In [29]:
import numpy as np
import matplotlib.pyplot as plt
from Pad import PadZero

In [30]:
def Sigmoid(x):
    return (np.exp(x))/((np.exp(x)) + 1)
def SigmoidPrime(x):
    return Sigmoid(x) * (1 - Sigmoid(x))

In [31]:
def Empty(x):
    return x

In [32]:
def feedForwardConv(prevA,y,w,b,hyperparameters, activationFunction = Sigmoid):
    (m, oldNH, oldNW, nC) = prevA.shape
    (f,f,nC) = w.shape
    stride = hyperparameters["stride"]
    pad = hyperparameters["padding"]
    
    newNH = int((oldNH - f + 2*pad)/stride) + 1
    newNW = int((oldNH - f + 2*pad)/stride) + 1
    
    z = np.zeros((m,newNH, newNW, nC))
    a = np.zeros((m,newNH, newNW, nC))
    
    if (pad != 0):
        prevA = PadZero(prevA, pad)
    
    for m1 in range(m):
        for i,h1 in enumerate(range(0,newNH,stride)):
            for j,w1 in enumerate(range(0,newNW, stride)):
                for c in range(nC):
                    z[m1,i,j,c] = np.sum((prevA[m1,h1:h1+f,w1:w1+f,c] * w[...,c]) + b)#[...,с] нужно, чтобы размерности совпали
                    a[m1,i,j,c] = activationFunction(z[m1,i,j,c])

    assert (m,newNH,newNW,nC) == a.shape
    
    cache = (a,y, w, b, hyperparameters)
    
    return a, cache

In [33]:
def backPropagationConv(cache, delta, activationFunctionPrime = SigmoidPrime):
    (a,y,w,b,hyperparameters) = cache[0]
    (m,nH,nW,nC) = a.shape
    (f,f,nC) = w.shape
    stride = hyperparameters["stride"]
    pad = hyperparameters["padding"]
    nablaW = np.zeros(w.shape)
    
    for m1 in range(m):
        for i,h1 in enumerate(range(0,nH-f+1,stride)):
            for j,w1 in enumerate(range(0,nW-f+1,stride)):
                for c1 in range(nC):
                    '''
                    print("\n\nm1: {}\ni:{} h1:{}\nj:{} w1:{}\nc1:{}".format(m1,i,h1,j,w1,c1))
                    print("W1: \n",nablaW[:,:,c1])
                    print("a: \n", a[m1,h1:h1+f,w1:w1+f,c1])
                    print("delta: \n",delta[h1:h1+f,w1:w1+f])
                    '''
                    newW = a[m1,h1:h1+f,w1:w1+f,c1] * delta[h1:h1+f,w1:w1+f]
                    nablaW[:,:,c1] += newW
                    '''
                    print("W2: \n",newW)
                    print("W: \n",nablaW[:,:,c1])
                    '''
    return nablaW

In [34]:
m = 1
nH = 4
nW = 4
nC = 1

sizeFilter = 3

stride = 1
pad = 1

y = np.array([1,0]).reshape(2,m)
#x = np.random.randint(0,16,(m,nH,nW,nC))
x = np.array([1,2,0,2,2,1,0,0,2,0,0,0,2,2,0,0]).reshape(m,nH,nW,nC)
print("x{}: \n{}".format(x.shape,x[0,:,:,0]))
w = np.ones((sizeFilter,sizeFilter,nC), dtype = np.int32)
b = 0
hyperparameters = {"stride": stride, "padding":pad}
caches2 = []
print("\nfilter/w{}: \n{}".format(w.shape,w[:,:,0]))


x(1, 4, 4, 1): 
[[1 2 0 2]
 [2 1 0 0]
 [2 0 0 0]
 [2 2 0 0]]

filter/w(3, 3, 1): 
[[1 1 1]
 [1 1 1]
 [1 1 1]]


In [35]:
a1, cache = feedForwardConv(x,y,w,b,hyperparameters, Empty)

print("a1{}: \n{}".format(a1.shape,a1[0,:,:,0]))


a1(1, 4, 4, 1): 
[[6. 6. 5. 2.]
 [8. 8. 5. 2.]
 [9. 9. 3. 0.]
 [6. 6. 2. 0.]]


In [36]:
caches2.append(cache)


In [37]:
delta = np.array([1,0,1,2,2,0,2,1,1,0,1,1,2,2,0,1]).reshape(4,4)
print("delta{}: \n{}".format(delta.shape,delta))

delta(4, 4): 
[[1 0 1 2]
 [2 0 2 1]
 [1 0 1 1]
 [2 2 0 1]]


In [38]:
nablaW = backPropagationConv(caches2, delta)

In [39]:
print("nablaW{}: \n{}".format(nablaW.shape,nablaW[...,0]))

nablaW(3, 3, 1): 
[[22. 15. 21.]
 [25. 13. 15.]
 [33. 15.  3.]]


In [40]:
(a2,y2,w2,b2,hyp2) = caches2[0]

In [41]:
[print(a) for a in a2]

[[[6.]
  [6.]
  [5.]
  [2.]]

 [[8.]
  [8.]
  [5.]
  [2.]]

 [[9.]
  [9.]
  [3.]
  [0.]]

 [[6.]
  [6.]
  [2.]
  [0.]]]


[None]